In [1]:
from functools import partial, reduce
from os import rename, makedirs
import urllib.request
import pandas as pd
import os.path
import shutil
import json
import csv
import re
import datetime

In [2]:
def gsheet_download_csv(id: str, target: str, url_template="https://docs.google.com/spreadsheets/d/{id}/export?format=csv"):
    url = url_template.format(id=id)
    return urllib.request.urlretrieve(url, target)

gsheet_download_csv('1kK1Yu6gz5kEWe_i0vamiGttkXUH5H90e', 'nomenclador.csv')
paises = pd.read_csv('./nomenclador.csv')
pais = {k:v for k,v in paises[['iso3', 'iso3_desc_fundar']].iloc}
codes_set = set(pais.keys())

lista_cambios = []
diccionario_cambios = {}

In [3]:
def match_relocate(src, target, pattern, file_list=None):
    files_to_move = file_list or os.listdir(src)

    pattern = re.compile(pattern)

    if not os.path.exists(target):
        makedirs(target)

    for filename in files_to_move:
        if pattern.match(filename):
            src_path = os.path.join(src, filename)
            dest_path = os.path.join(target, filename)

            shutil.move(src_path, dest_path)

    print(f"Files matching the pattern '{pattern}' moved to '{target}'.")

def compose2(f, g):
    def composed(*a, **kw):
        return f(g(*a, **kw))

    return composed

def compose(*fs):
    return reduce(compose2, fs)

def _chain(*fs):
    return compose(*reversed(fs))

def chain(*fs):
    lista_cambios.clear()
    return _chain(*fs)

def exportar_definitivo(archivo: str, folder:str, df: pd.DataFrame, nuevo_nombre=None):
    if not nuevo_nombre:
        nuevo_nombre = f"{folder}/definitivos/{archivo}_old.csv"

    original_file = f'{folder}/definitivos/{archivo}.csv'

    rename(original_file, nuevo_nombre)

    df.to_csv(original_file, 
              encoding='utf-8', 
              sep=',', 
              quoting=csv.QUOTE_ALL, 
              quotechar='"', 
              lineterminator='\n', 
              decimal='.', 
              index=False)
    
    return original_file, nuevo_nombre

def drop_col(df: pd.DataFrame, col, axis=1, lista_cambios = lista_cambios):
    lista_cambios.append(f"Se borró la columna {col}")
    return df.drop(col, axis=axis)

def drop_colx(col, axis=1):
    return lambda df: drop_col(df, col, axis=axis)

def normalize_countries(df: pd.DataFrame, code_col:str, new_col:str, desc_col:str, map: dict, lista_cambios = lista_cambios):
    codes_set = set(map.keys())
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    diff_set = set(df[code_col].dropna()) - codes_set
    if len(diff_set)>0: 
        diff_str = ", ".join([str(x) for x in list(diff_set)])
        string = f"ERROR: Los códigos {diff_str} no se encontraron en el nomenclador vigente al {today} en la columna {code_col}"
        lista_cambios.append(string)
    else:
        if (new_col and desc_col):
            df[new_col] = df[code_col].apply(map.__getitem__)
            df = df.drop(desc_col, axis=1)
            string = f"Usando los valores de la columna {code_col}, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al {today}, creando la columna {new_col} y borrando la columna {desc_col}"
        elif desc_col:
            df[desc_col] = df[code_col].apply(map.__getitem__)
            string = f"Usando los valores de la columna {code_col}, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al {today}, cambiando los valores de la columna {desc_col}"
        else: 
            string = f"Usando los valores de la columna {code_col}, se verificó que todos los códigos se encuentran en el nomenclador vigente al {today}"
        lista_cambios.append(string)
    return df

def normalize_countriesx(code_col, new_col, desc_col, map):
    return lambda df: normalize_countries(df, code_col=code_col, new_col=new_col, desc_col=desc_col, map=map)

def wide_to_long(df: pd.DataFrame, primary_keys, value_name='valor', var_name='indicador', lista_cambios=lista_cambios):
    cols = ", ".join(df.columns)
    new_cols = ", ".join(primary_keys + [value_name, var_name])
    string = f"Long estricto: [{cols}] -> [{new_cols}]"
    lista_cambios.append(string)
    return df.melt(id_vars=primary_keys, value_name=value_name, var_name=var_name)

def wide_to_longx(primary_keys, value_name='valor', var_name='indicador'):
    return lambda df: wide_to_long(df, primary_keys, value_name=value_name, var_name=var_name)

def rename_cols(df: pd.DataFrame, map, lista_cambios=lista_cambios):
    map_string = ", ".join(f"{old_col}:{new_col}" for old_col, new_col in map.items())
    string = f"Se renombraron las columnas: {map_string}"
    lista_cambios.append(string)
    df = df.rename(columns=map)
    return df

def rename_colsx(map):
    return lambda df: rename_cols(df, map)

def replace_value(df:pd.DataFrame, col:str, curr_value:str, new_value:str, lista_cambios=lista_cambios):
    string = f"Se imputó el valor {new_value} en la columna {col} cuando {col}=={curr_value}"
    lista_cambios.append(string)
    df = df.replace({col: curr_value}, new_value)
    return df

def replace_valuex(col, curr_value, new_value):
    return lambda df: replace_value(df=df, col=col, curr_value=curr_value, new_value=new_value)

def sort_vals_asc(df:pd.DataFrame, prim_keys:list, lista_cambios=lista_cambios):
    lista_cambios.append(f"Ordené las filas por las columnas: {', '.join(prim_keys)}")
    return df.sort_values(by=prim_keys).reset_index(drop=True)

def sort_vals_ascx(prim_keys):
    return lambda df: sort_vals_asc(df=df, prim_keys=prim_keys)

AGROPE_g1.csv

In [5]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 1

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio


,anio,participacion_agro
0,1935,0.21
1,1936,0.21
2,1937,0.23
3,1938,0.20
4,1939,0.19


In [6]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)


pipeline = chain(
    to_long(primary_keys),
    sort_vals_ascx(primary_keys)
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'region_code': 'cod_area'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)


diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,indicador,valor
0,1935,participacion_agro,0.21
1,1936,participacion_agro,0.21
2,1937,participacion_agro,0.23
3,1938,participacion_agro,0.20
4,1939,participacion_agro,0.19


In [7]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio']}

In [8]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g1.csv',
 '../output/AGROPE1/definitivos/AGROPE_g1_old.csv')

AGROPE_g2.csv

In [9]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 2

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio


,anio,pbi_agro_pcons
0,1875,971
1,1876,1139
2,1877,1302
3,1878,1237
4,1879,1330


In [10]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys),
    sort_vals_ascx(primary_keys)
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'region_code': 'cod_area'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,indicador,valor
0,1875,pbi_agro_pcons,971
1,1876,pbi_agro_pcons,1139
2,1877,pbi_agro_pcons,1302
3,1878,pbi_agro_pcons,1237
4,1879,pbi_agro_pcons,1330


In [11]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio']}

In [12]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g2.csv',
 '../output/AGROPE1/definitivos/AGROPE_g2_old.csv')

AGROPE_g3.csv

In [13]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 3

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio cuenta


,anio,cuenta,participacion_pbi
0,2004,Agricultura,0.05
1,2005,Agricultura,0.04
2,2006,Agricultura,0.04
3,2007,Agricultura,0.05
4,2008,Agricultura,0.05


In [14]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys),
    sort_vals_ascx(primary_keys)
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'region_code': 'cod_area'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,cuenta,indicador,valor
0,2004,Agricultura,participacion_pbi,0.050
1,2004,Otros,participacion_pbi,0.010
2,2004,Pecuario,participacion_pbi,0.020
3,2004,Total,participacion_pbi,0.081
4,2005,Agricultura,participacion_pbi,0.040


In [15]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta']}

In [16]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g3.csv',
 '../output/AGROPE1/definitivos/AGROPE_g3_old.csv')

AGROPE_g4.csv

In [17]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 4

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3c anio


,iso3c,anio,va_agro_sobre_pbi,pais
0,HIC,2022,NaN,Países de ingreso alto
1,HIC,2021,1.276669,Países de ingreso alto
2,HIC,2020,1.285265,Países de ingreso alto
3,HIC,2019,1.217419,Países de ingreso alto
4,HIC,2018,1.267516,Países de ingreso alto


In [23]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys + ['pais']),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3c,anio,pais,indicador,valor
0,ARG,1960,Argentina,va_agro_sobre_pbi,NaN
1,ARG,1961,Argentina,va_agro_sobre_pbi,NaN
2,ARG,1962,Argentina,va_agro_sobre_pbi,NaN
3,ARG,1963,Argentina,va_agro_sobre_pbi,NaN
4,ARG,1964,Argentina,va_agro_sobre_pbi,NaN


In [24]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio']}

In [25]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g4.csv',
 '../output/AGROPE1/definitivos/AGROPE_g4_old.csv')

AGROPE_g5.csv

In [26]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 5

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3c anio


,iso3c,pais,anio,va_agro_sobre_pbi
0,MIC,Ingreso mediano,2021,8.890954
1,WLD,Media mundial,2021,4.304093
2,ARG,Argentina,2021,7.131541
3,BOL,Bolivia,2021,12.922253
4,BRA,Brasil,2021,7.486921


In [27]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys + ['pais']),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3c,anio,pais,indicador,valor
0,ARG,2021,Argentina,va_agro_sobre_pbi,7.131541
1,BOL,2021,Bolivia,va_agro_sobre_pbi,12.922253
2,BRA,2021,Brasil,va_agro_sobre_pbi,7.486921
3,CHL,2021,Chile,va_agro_sobre_pbi,3.645349
4,COL,2021,Colombia,va_agro_sobre_pbi,7.635340


In [28]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [29]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g5.csv',
 '../output/AGROPE1/definitivos/AGROPE_g5_old.csv')

AGROPE_g6.csv

In [30]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 6

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio cod_pcia


,anio,nom_pcia,valor,cod_pcia
0,2004,Ciudad de Buenos Aires,0.0021,2
1,2005,Ciudad de Buenos Aires,0.0020,2
2,2006,Ciudad de Buenos Aires,0.0017,2
3,2007,Ciudad de Buenos Aires,0.0018,2
4,2008,Ciudad de Buenos Aires,0.0018,2


In [38]:
# df2 = df.copy()
# df2.columns = ['anio','nom_pcia','value','cod_pcia']
df.melt(id_vars=['anio','cod_pcia','nom_pcia'], value_name="_valor", var_name="indicador")

,anio,cod_pcia,nom_pcia,indicador,_valor
0,2004,2,Ciudad de Buenos Aires,valor,0.0021
1,2005,2,Ciudad de Buenos Aires,valor,0.0020
2,2006,2,Ciudad de Buenos Aires,valor,0.0017
3,2007,2,Ciudad de Buenos Aires,valor,0.0018
4,2008,2,Ciudad de Buenos Aires,valor,0.0018
...,...,...,...,...,...
427,2017,94,Tierra del Fuego,valor,0.0026
428,2018,94,Tierra del Fuego,valor,0.0029
429,2019,94,Tierra del Fuego,valor,0.0031
430,2020,94,Tierra del Fuego,valor,0.0034


In [40]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys + ['nom_pcia'], value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,cod_pcia,nom_pcia,indicador,valor
0,2004,2,Ciudad de Buenos Aires,valor,0.0021
1,2004,6,Buenos Aires,valor,0.0889
2,2004,10,Catamarca,valor,0.0501
3,2004,14,Córdoba,valor,0.2114
4,2004,18,Corrientes,valor,0.0660


In [41]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [42]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g6.csv',
 '../output/AGROPE1/definitivos/AGROPE_g6_old.csv')

AGROPE_g7.csv

In [43]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 7

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

cultivo campania


,cultivo,campania,q_total,share
0,Algodón,2021/22,1115510,0.777280
1,Alpiste,2021/22,36114,0.025164
2,Arroz,2021/22,1222426,0.851779
3,Arveja,2021/22,296957,0.206918
4,Avena,2021/22,723154,0.503889


In [44]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,cultivo,campania,indicador,valor
0,Algodón,2021/22,q_total,1.115510e+06
1,Algodón,2021/22,share,7.772804e-01
2,Alpiste,2021/22,q_total,3.611400e+04
3,Alpiste,2021/22,share,2.516401e-02
4,Arroz,2021/22,q_total,1.222426e+06


In [45]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [46]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g7.csv',
 '../output/AGROPE1/definitivos/AGROPE_g7_old.csv')

AGROPE_g8.csv

In [47]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 8

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

cultivo campania


,cultivo,campania,q_total
0,Maíz,1969/70,9358372.0
1,Maíz,1970/71,9928194.0
2,Maíz,1971/72,5858318.0
3,Maíz,1972/73,9698405.0
4,Maíz,1973/74,9898570.0


In [48]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,cultivo,campania,indicador,valor
0,Maíz,1969/70,q_total,9358372.0
1,Maíz,1970/71,q_total,9928194.0
2,Maíz,1971/72,q_total,5858318.0
3,Maíz,1972/73,q_total,9698405.0
4,Maíz,1973/74,q_total,9898570.0


In [49]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [50]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g8.csv',
 '../output/AGROPE1/definitivos/AGROPE_g8_old.csv')

AGROPE_g9.csv

In [51]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 9

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

cultivo campania


,cultivo,campania,supsem_total
0,Maíz,1969/70,4663760.0
1,Maíz,1970/71,4990990.0
2,Maíz,1971/72,4437630.0
3,Maíz,1972/73,4249455.0
4,Maíz,1973/74,4132500.0


In [52]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,cultivo,campania,indicador,valor
0,Maíz,1969/70,supsem_total,4663760.0
1,Maíz,1970/71,supsem_total,4990990.0
2,Maíz,1971/72,supsem_total,4437630.0
3,Maíz,1972/73,supsem_total,4249455.0
4,Maíz,1973/74,supsem_total,4132500.0


In [53]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [54]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g9.csv',
 '../output/AGROPE1/definitivos/AGROPE_g9_old.csv')

AGROPE_g10.csv

In [55]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 10

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

cultivo campania


,cultivo,campania,rindes_avg
0,Maíz,1969/70,2330.502874
1,Maíz,1970/71,2442.963527
2,Maíz,1971/72,1862.165290
3,Maíz,1972/73,2720.995936
4,Maíz,1973/74,2840.376476


In [56]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,cultivo,campania,indicador,valor
0,Maíz,1969/70,rindes_avg,2330.502874
1,Maíz,1970/71,rindes_avg,2442.963527
2,Maíz,1971/72,rindes_avg,1862.165290
3,Maíz,1972/73,rindes_avg,2720.995936
4,Maíz,1973/74,rindes_avg,2840.376476


In [57]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [59]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g10.csv',
 '../output/AGROPE1/definitivos/AGROPE_g10_old.csv')

AGROPE_g11.csv

In [60]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 11

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

m49_code_unsd anio


,m49_code_unsd,anio,rindes_maiz,pais,rindes_maiz_ma5
0,32,1961,1.7672,argentina,NaN
1,32,1962,1.8936,argentina,NaN
2,32,1963,1.6481,argentina,NaN
3,32,1964,1.8010,argentina,NaN
4,32,1965,1.6785,argentina,1.75768


In [79]:
m49_cod = paises[['m49_code_unsd', 'iso3_desc_fundar']].dropna(how="any").sort_values(by="m49_code_unsd").reset_index(drop=True)
m49_cod['m49_code_unsd'] = m49_cod['m49_code_unsd'].astype('int')
pais_m49_cod = {k:v for k,v in m49_cod.iloc}
codes_set_m49_cod = set(pais_m49_cod.keys())


In [85]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais_m49_cod)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='m49_code_unsd', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys + ['pais']),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,m49_code_unsd,anio,pais,indicador,valor
0,32,1961,argentina,rindes_maiz,1.7672
1,32,1961,argentina,rindes_maiz_ma5,NaN
2,32,1962,argentina,rindes_maiz,1.8936
3,32,1962,argentina,rindes_maiz_ma5,NaN
4,32,1963,argentina,rindes_maiz,1.6481


In [86]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [ ]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g4.csv',
 '../output/AGROPE1/definitivos/AGROPE_g4_old.csv')

AGROPE_g11.csv

In [5]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 11

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

m49_code_unsd anio


,m49_code_unsd,anio,rindes_maiz,pais,rindes_maiz_ma5
0,32,1961,1.7672,argentina,NaN
1,32,1962,1.8936,argentina,NaN
2,32,1963,1.6481,argentina,NaN
3,32,1964,1.8010,argentina,NaN
4,32,1965,1.6785,argentina,1.75768


In [10]:
df[df.m49_code_unsd==159]

,m49_code_unsd,anio,rindes_maiz,pais,rindes_maiz_ma5
124,159,1991,4.5799,china,NaN
125,159,1992,4.5347,china,NaN
126,159,1993,4.9641,china,NaN
127,159,1994,4.6951,china,NaN
128,159,1995,4.9177,china,4.73830
129,159,1996,5.2039,china,4.86310
130,159,1997,4.3901,china,4.83418
131,159,1998,5.2686,china,4.89508
132,159,1999,4.9458,china,4.94522
133,159,2000,4.5992,china,4.88152


In [6]:
m49_cod = paises[['m49_code_unsd', 'iso3_desc_fundar']].dropna(how="any").sort_values(by="m49_code_unsd").reset_index(drop=True)
m49_cod['m49_code_unsd'] = m49_cod['m49_code_unsd'].astype('int')
pais_m49_cod = {k:v for k,v in m49_cod.iloc}
codes_set_m49_cod = set(pais_m49_cod.keys())


In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais_m49_cod)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='m49_code_unsd', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys + ['pais']),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,m49_code_unsd,anio,pais,indicador,valor
0,32,1961,argentina,rindes_maiz,1.7672
1,32,1961,argentina,rindes_maiz_ma5,NaN
2,32,1962,argentina,rindes_maiz,1.8936
3,32,1962,argentina,rindes_maiz_ma5,NaN
4,32,1963,argentina,rindes_maiz,1.6481


In [ ]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-11, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [ ]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g4.csv',
 '../output/AGROPE1/definitivos/AGROPE_g4_old.csv')

In [ ]:
match_relocate(f'{folder}/definitivos', f'{folder}/old', '.*old.*')

In [5]:
import json
from datetime import datetime

today = 

with open(f'../output/AGROPE1/definitivos/paso_a_definitivos_.json', 'w') as fp:
    json.dump(diccionario_cambios, fp, indent=4, ensure_ascii=False)
